## 4.6 Merging and exporting data - Part 1

### This script contains the following points:

#### 1. Check output and dimensions of df_ords and df_ords_prior
#### 2. Do consistency checks on df_ords_prior
#### 3. Merge df_ords and df_ords_prior

### The following dataframes were manipulated/created:

#### a) 1 column deleted from 'df_ords' (extra index). Exported as 'ords_checked.csv' (overwritten previous df_ords_checked)
#### b) new dataset created: 'orders_products_combined.pkl' (outer join of 'ords_checked' and 'orders_products_prior')

In [1]:
# Import librabries

import pandas as pd
import numpy as np
import os

In [2]:
# Import dataframes

path = r'C:\Users\bruna\Career Foundry\08-2023 Instacart Basket Analysis'

In [3]:
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'ords_checked.csv'))

In [4]:
df_ords_prior = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'orders_products_prior.csv'))


## 1. Check output and dimensions of df_ords and df_ords_prior

In [5]:
# Check the output of df_ords_prior and df_ords

df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [6]:
df_ords.tail()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,time_of_order,days_since_prior_order,first_order_flag
3421078,3421078,2266710,206209,10,5,18,29.0,False
3421079,3421079,1854736,206209,11,4,10,30.0,False
3421080,3421080,626363,206209,12,1,12,18.0,False
3421081,3421081,2977660,206209,13,1,12,7.0,False
3421082,3421082,272231,206209,14,6,14,30.0,False


    df_ords_prior and df_ords have column 'order_id' in common --> This can be used to merge the two dataframes
    df_ords has 2 columns that can be deleted, as they are indexes that were accidentally created (unnamned:0.1 and unnamed:0)

In [8]:

# Dropping unnecessary columns in df_ords

df_ords= df_ords.drop(columns=['Unnamed: 0'])

In [9]:
df_ords.head()

,order_id,user_id,order_number,orders_day_of_week,time_of_order,days_since_prior_order,first_order_flag
0,2539329,1,1,2,8,NaN,True
1,2398795,1,2,3,7,15.0,False
2,473747,1,3,3,12,21.0,False
3,2254736,1,4,4,7,29.0,False
4,431534,1,5,4,15,28.0,False


In [10]:
# Check dimensions of df_ords_prior and df_ords

df_ords_prior.shape

(32434489, 4)

In [11]:
df_ords.shape

(3421083, 7)


## 2. Do consistency checks on df_ords_prior

In [12]:
# Check for mixed type columns

for col in df_ords_prior.columns.tolist():
    weird = (df_ords_prior[[col]].applymap(type) != df_ords_prior[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_ords_prior[weird]) > 0:
        print (col)

    No mixed type columns founf

In [13]:
# Looking for columns with missing values

df_ords_prior.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

    No columns with missing values found

In [14]:
# Looking duplicate values

df_dups = df_ords_prior[df_ords_prior.duplicated()]

In [15]:
df_dups

,order_id,product_id,add_to_cart_order,reordered


    No duplicated rows found
    No changes required in df_ords_prior


## 3. Merge df_ords and df_ords_prior

In [16]:
# Merge df_ords_prior with df_ords

df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)

In [17]:
# Run frequency check of _merge column

df_merged_large['_merge'].value_counts()

both          32434489
left_only            0
right_only           0
Name: _merge, dtype: int64

In [18]:
df_merged_large.head()

,order_id,user_id,order_number,orders_day_of_week,time_of_order,days_since_prior_order,first_order_flag,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,both
3,2539329,1,1,2,8,NaN,True,26088,4,0,both
4,2539329,1,1,2,8,NaN,True,26405,5,0,both


In [19]:
#  Double-check merge rates using an outer join

df_merged_outer_test = df_ords.merge(df_ords_prior, on = 'order_id', how = 'outer', indicator = True)

In [20]:
df_merged_outer_test['_merge'].value_counts()

both          32434489
left_only       206209
right_only           0
Name: _merge, dtype: int64

In [21]:
del df_merged_outer_test # deleting dataframe to free up memory (dataframe not needed)

In [22]:
# Check dimensions of df_merged_large

df_merged_large.shape

(32434489, 11)

## 3. Exporting Dataframes

In [23]:
# Exporting 'ords_checked'

df_ords.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'ords_checked.csv'))

In [24]:
# Exporting 'orders_products_combined.pkl'

df_merged_large.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl'))